In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import scipy.stats as stats

In [ ]:
crimeHousing = pd.read_csv('crime-housing-austin-2015.csv')
zipCodes = pd.read_csv('AustinZipCodes.csv')
pd.set_option('display.max_columns', None)

# Crime per Month

# Housing vs Crime

In [ ]:
crimeAustin = crimeHousing[crimeHousing.Zip_Code_Housing.notna()]
crimeAustin = crimeHousing[['Highest_NIBRS_UCR_Offense_Description','Zip_Code_Crime']].reset_index()

In [ ]:
chart = sns.countplot(x='Highest_NIBRS_UCR_Offense_Description', data=crimeAustin)
plt.xticks(rotation=15)
chart.set_title('Amount of Crimes')
chart.set_xlabel('Crime')
chart.set_ylabel('Amount')
plt.savefig('overall-crime-stats.pdf')

In [ ]:
theft = crimeAustin[(crimeAustin.Highest_NIBRS_UCR_Offense_Description == "Theft")]
theft = theft[theft.Zip_Code_Crime.notna()]
theft.Zip_Code_Crime = theft.Zip_Code_Crime.astype('int')
theftCrime = theft.groupby('Zip_Code_Crime').agg({'Highest_NIBRS_UCR_Offense_Description': 'count'})
theftCrime.sort_values(by=['Highest_NIBRS_UCR_Offense_Description'])
ax = sns.scatterplot(data=theftCrime)
ax.set_title('Thefts in Zip Code')
ax.set_xlabel('Zip Code of Crime')
ax.set_ylabel('Amount of Thefts')

In [ ]:
housing = crimeHousing[['Zip_Code_Housing', 'Medianhouseholdincome']].reset_index()
housing = housing[housing.Zip_Code_Housing.notna()]
housing.Zip_Code_Housing = housing.Zip_Code_Housing.astype('int')
housing['Medianhouseholdincome'] = housing['Medianhouseholdincome'].str.replace('$', '').astype('float')
housing = housing.groupby('Zip_Code_Housing').agg({'Medianhouseholdincome': 'mean'})
housing.sort_values(by='Zip_Code_Housing')
#display(stats.pearsonr(housing.Medianhouseholdincome, housing.Zip_Code_Housing))
ax = sns.scatterplot(data=housing)
ax.set_title('Household income per Zip Code')
ax.set_xlabel('Zip Code of Housing')
ax.set_ylabel('Median Household Income')

In [ ]:
pearTheft = crimeHousing[(crimeAustin.Highest_NIBRS_UCR_Offense_Description == "Theft")]
pearTheft = pearTheft[pearTheft.Zip_Code_Housing.notna()]
pearTheft = pearTheft[pearTheft.Zip_Code_Crime.notna()]
pearTheft.Zip_Code_Housing = pearTheft.Zip_Code_Housing.astype('int')
pearTheft.Zip_Code_Crime = pearTheft.Zip_Code_Crime.astype('int')
pearTheft = pearTheft[['Highest_NIBRS_UCR_Offense_Description', 'Zip_Code_Crime','Zip_Code_Housing', 'Medianhouseholdincome']].reset_index()
pearTheft['Medianhouseholdincome'] = pearTheft['Medianhouseholdincome'].str.replace('$', '').astype('float')
display(stats.pearsonr(pearTheft.Zip_Code_Housing, pearTheft.Zip_Code_Crime))
display(stats.pearsonr(pearTheft.Zip_Code_Housing, pearTheft.Medianhouseholdincome))
display(stats.pearsonr(pearTheft.Zip_Code_Crime, pearTheft.Medianhouseholdincome))